In [ ]:
!cat /proc/cpuinfo

In [ ]:
import sys
path_workspace = '/content/r2_learning_calibration'
path_source = '/content/drive/MyDrive/RAVEN Calibration/experiments/recorded_trajs'
sys.path.append(path_source)

In [ ]:
!mkdir '/content/r2_learning_calibration'
!mkdir '/content/r2_learning_calibration/data'

# !unzip -q "/content/drive/MyDrive/RAVEN Calibration/experiments/recorded_trajs/record_1_different_directions.zip" -d '/content/r2_learning_calibration/data'

# !unzip -q "/content/drive/MyDrive/RAVEN Calibration/experiments/recorded_trajs/record_2_diff_sparsity.zip" -d '/content/r2_learning_calibration/data'

!unzip -q "/content/drive/MyDrive/RAVEN Calibration/experiments/recorded_trajs/record_3_time_decay_unloaded.zip" -d '/content/r2_learning_calibration/data'

# !unzip -q "/content/drive/MyDrive/RAVEN Calibration/experiments/recorded_trajs/record_3_time_decay_idle.zip" -d '/content/r2_learning_calibration/data'

# !unzip -q "/content/drive/MyDrive/RAVEN Calibration/experiments/recorded_trajs/record_3_time_decay_unloaded.zip" -d '/content/r2_learning_calibration/data'

# !unzip -q "/content/drive/MyDrive/RAVEN Calibration/experiments/recorded_trajs/record_4_home_decay_no_home_in_training.zip" -d '/content/r2_learning_calibration/data'

# !unzip -q "/content/drive/MyDrive/RAVEN Calibration/experiments/recorded_trajs/record_4_home_decay_with_home_in_training.zip.zip" -d '/content/r2_learning_calibration/data'

In [ ]:
import os
import time
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import ElasticNet

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# import data
path_data = '/content/r2_learning_calibration/data/'

combine_testset = False
train_on_err = False
save_rst = True  # if true, prediction result will be saved as csv

# EXP1 Different Direction
# # file_train_list_ = ['data_record_x_05.csv', 'data_record_yz_03.csv','data_record_yz_025.csv']
file_train_list_ = ['data_record_xyz_05.csv','data_record_xyz_03.csv','data_record_xyz_025.csv']

# # file_test_list_ = ['data_record_rand1200_x.csv', 'data_record_rand1200_y.csv', 'data_record_rand1200_x.csv','data_record_rand1200_x.csv', 'data_record_rand1200_x.csv']
file_test_list_ = [ 'data_record_rand1200_xyz.csv']

# EXP2 Different Sparsity
# # file_train_list_ = ['data_record_yz_05_1.csv','data_record_yz_03_1.csv','data_record_yz_025_1.csv', 'data_record_yz_02_1.csv', 'data_record_yz_016_1.csv']
# file_train_list_ = ['data_record_yz_03_5.csv', 'data_record_yz_02_3.csv']
# # file_test_list_ = ['data_record_rand1200_05.csv', 'data_record_rand1200_03.csv', 'data_record_rand1200_025.csv','data_record_rand1200_02.csv', 'data_record_rand1200_016.csv']
# file_test_list_ = ['data_record_rand1200_03.csv', 'data_record_rand1200_02.csv']

# EXP3 Time decay
# # file_train_list_ = ['data_record_yz_05_1.csv', 'data_record_yz_03_1.csv', 'data_record_yz_025_1.csv', 'data_record_yz_02_1.csv', 'data_record_yz_016_1.csv']
# file_train_list_ = ['data_record_yz_016_1.csv']
# file_test_list_ = ['data_record_rand1200_h0.csv', 'data_record_rand1200_h1.csv', 'data_record_rand1200_h2.csv', 'data_record_rand1200_h3.csv', 'data_record_rand1200_h4.csv', 'data_record_rand1200_h5.csv']

# # file_train_list_ = ['data_record_yz_05.csv', 'data_record_yz_03.csv', 'data_record_yz_025.csv', 'data_record_yz_02.csv', 'data_record_yz_016.csv']
# file_train_list_ = ['data_record_yz_05.csv', 'data_record_yz_03.csv', 'data_record_yz_025.csv']
# file_test_list_ = ['data_record_rand1200_idle_h0.csv', 'data_record_rand1200_idle_h5.csv']

# file_train_list_ = ['data_record_yz_05.csv', 'data_record_yz_03.csv', 'data_record_yz_025.csv', 'data_record_yz_02.csv', 'data_record_yz_016.csv']
# file_train_list_ = ['data_record_yz_05.csv', 'data_record_yz_03.csv', 'data_record_yz_025.csv']
# file_test_list_ = ['data_record_rand1200_h0.csv', 'data_record_rand1200_h1.csv', 'data_record_rand1200_h2.csv', 'data_record_rand1200_h3.csv', 'data_record_rand1200_h4.csv', 'data_record_rand1200_h5.csv']

# EXP4 Homing decay - NO homing in training
# file_train_list_ = ['data_record_yz_05.csv', 'data_record_yz_03.csv', 'data_record_yz_025.csv', 'data_record_yz_02.csv', 'data_record_yz_016.csv']
# file_train_list_ = ['data_record_yz_03.csv', 'data_record_yz_016.csv']
# file_test_list_ = ['data_record_rand1200_hm0.csv', 'data_record_rand1200_hm1.csv', 'data_record_rand1200_hm2.csv', 'data_record_rand1200_hm3.csv', 'data_record_rand1200_hm4.csv', 'data_record_rand1200_hm5.csv']

# EXP4 Homing decay - WITH homing in training
# file_train_list_ = ['data_record_yz_05_hm0.csv', 'data_record_yz_03_hm1.csv', 'data_record_yz_025_hm1.csv', 'data_record_yz_02_hm1.csv', 'data_record_yz_016_hm0.csv']
# file_train_list_ = [ 'data_record_yz_03_hm1.csv', 'data_record_yz_016_hm0.csv']
# file_test_list_ = ['data_record_rand1200_hm0.csv', 'data_record_rand1200_hm1.csv', 'data_record_rand1200_hm2.csv', 'data_record_rand1200_hm3.csv', 'data_record_rand1200_hm4.csv', 'data_record_rand1200_hm5.csv']

# left arm only ----------------------------
feature_list = [
                np.arange(12,13), # time stamp
                np.arange(13,21), # jpos
                np.arange(29,30), # run level
                np.arange(30,31), # sub level
                np.arange(31,32), # last_seq
                np.arange(32,33), # type
                np.arange(34,37), # pos
                np.arange(40,49), # ori
                np.arange(58,67), # ord_d
                np.arange(76,79), # pos_d
                np.arange(82,98), # enc_val
                np.arange(98,106), # dac_val
                np.arange(114,122), # tau
                np.arange(130,133),np.arange(133,138), # mpos
                np.arange(146,154), #  mvel
                np.arange(162,170), # jvel
                np.arange(178,186), # mpos_d
                np.arange(194,202), # jpos_d
                np.arange(210,211), # grasp_d
                np.arange(212,228), # enc_offset
                np.arange(228,234), # jac_vel
                np.arange(240,246) # jac_f
                ]

print(feature_list.shape)

noise_type = 'no_noise'
noise_list = [np.arange(82,98), # enc_val
              np.arange(212,228) # enc_offset
              ]

feature_select = np.array([])
for feature in feature_list:
  feature_select = np.hstack([feature_select, feature])
feature_select = feature_select.astype(int)
feature_dim = feature_select.size

noise_select = np.array([])
for noise in noise_list:
  noise_select = np.hstack([noise_select, noise])
noise_select = noise_select.astype(int)


In [ ]:

# Load training data
first_call = True
for file_training in file_train_list_:
  data = np.loadtxt(path_data + file_training, delimiter = ',')
  if first_call:
    data_train = data
    first_call = False
  else:
    data_train = np.vstack((data_train, data))

# Load testing data
if combine_testset == True:
  first_call = True
  for file_test in file_test_list_:
    data = np.loadtxt(path_data + file_test, delimiter = ',')
    if first_call:
      data_test = data
      first_call = False
    else:
      data_test = np.vstack((data_test, data))
  data_test_list = [data_test]
else:
  data_test_list = []
  for file_test in file_test_list_:
    data = np.loadtxt(path_data + file_test, delimiter = ',')
    data_test_list.append(data)

print('Training data loaded, shape:')
print(data_train.shape)
print('Test data loaded, shape:')
for data_test in data_test_list:
  print(data_test.shape)

rad2deg = 180.0/np.pi
deg2rad = np.pi/180.0

# load the data for visualization only, joint 1 and 2 will be in deg, joint 3 in m
time_line = data_train[:,0]
ext_jpos_1 = data_train[:,1] *rad2deg   # joint pose from external joint encoders
ext_jpos_2 = data_train[:,2] *rad2deg
ext_jpos_3 = data_train[:,3]
rs_jpos_1 = data_train[:,13]            # joint pose from ravenstate
rs_jpos_2 = data_train[:,14]
rs_jpos_3 = data_train[:,15] *deg2rad

rs_jpos_1_lc = rs_jpos_1 + (np.mean(ext_jpos_1)-np.mean(rs_jpos_1))
rs_jpos_2_lc = rs_jpos_2 + (np.mean(ext_jpos_2)-np.mean(rs_jpos_2))
rs_jpos_3_lc = rs_jpos_3 + (np.mean(ext_jpos_3)-np.mean(rs_jpos_3))

bias_jpos_1 = np.mean(ext_jpos_1)-np.mean(rs_jpos_1)
bias_jpos_2 = np.mean(ext_jpos_2)-np.mean(rs_jpos_2)
bias_jpos_3 = np.mean(ext_jpos_3)-np.mean(rs_jpos_3)


fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(ext_jpos_1, ext_jpos_2, ext_jpos_3)
ax.plot3D(rs_jpos_1, rs_jpos_2, rs_jpos_3)
plt.legend(('ext_enc','jpos_raven_state'))
plt.title('Traning 3D traj')

fig = plt.figure()
plt.plot(time_line, ext_jpos_1)
plt.plot(time_line, rs_jpos_1, 'r--')
plt.legend(('ext_enc','jpos_raven_state'))
plt.title('Joint 1 raw')

fig = plt.figure()
plt.plot(time_line, ext_jpos_2)
plt.plot(time_line, rs_jpos_2, 'r--')
plt.legend(('ext_enc','jpos_raven_state'))
plt.title('Joint 2 raw')

fig = plt.figure()
plt.plot(time_line, ext_jpos_3)
plt.plot(time_line, rs_jpos_3, 'r--')
plt.legend(('ext_enc','jpos_raven_state'))
plt.title('Joint 3 raw')

fig = plt.figure()
plt.plot(time_line, ext_jpos_1)
plt.plot(time_line, rs_jpos_1_lc, 'r--')
plt.legend(('ext_enc','jpos_raven_state'))
plt.title('Joint 1 linear correct')

fig = plt.figure()
plt.plot(time_line, ext_jpos_2)
plt.plot(time_line, rs_jpos_2_lc, 'r--')
plt.legend(('ext_enc','jpos_raven_state'))
plt.title('Joint 2 linear correct')

fig = plt.figure()
plt.plot(time_line, ext_jpos_3)
plt.plot(time_line, rs_jpos_3_lc, 'r--')
plt.legend(('ext_enc','jpos_raven_state'))
plt.title('Joint 3 linear correct')

fig = plt.figure()
plt.plot(data_train[:,12] )
# plt.legend(('ext_enc','jpos_raven_state'))
plt.title('Time')


In [ ]:
# Regression first

np.random.seed(17)


start_time = time.time()


x_train = data_train[:, feature_select]
if 12 in feature_select:
  x_train[:,0] = data_train[:, 0] # change the time stamp to relative time (start with 0)
y_train = data_train[:, 1:4]

ext_jpos_1 = data_train[:,1] *rad2deg   # joint pose from external joint encoders
ext_jpos_2 = data_train[:,2] *rad2deg
ext_jpos_3 = data_train[:,3]
rs_jpos_1 = data_train[:,13]            # joint pose from ravenstate
rs_jpos_2 = data_train[:,14]
rs_jpos_3 = data_train[:,15] *deg2rad

rs_jpos_1_lc = rs_jpos_1 + (np.mean(ext_jpos_1)-np.mean(rs_jpos_1))
rs_jpos_2_lc = rs_jpos_2 + (np.mean(ext_jpos_2)-np.mean(rs_jpos_2))
rs_jpos_3_lc = rs_jpos_3 + (np.mean(ext_jpos_3)-np.mean(rs_jpos_3))

bias_jpos_1 = np.mean(ext_jpos_1)-np.mean(rs_jpos_1)
bias_jpos_2 = np.mean(ext_jpos_2)-np.mean(rs_jpos_2)
bias_jpos_3 = np.mean(ext_jpos_3)-np.mean(rs_jpos_3)

fig = plt.figure()
plt.plot(x_train[:,0])
# plt.legend(('ext_enc','jpos_raven_state'))
plt.title('Time')

if train_on_err:
  y_train[:,0] = ext_jpos_1 - rs_jpos_1
  y_train[:,1] = ext_jpos_2 - rs_jpos_2
  y_train[:,2] = ext_jpos_3 - rs_jpos_3
else:
  y_train[:,0] = ext_jpos_1
  y_train[:,1] = ext_jpos_2
  y_train[:,2] = ext_jpos_3

scaler_x_train = StandardScaler()
x_train_scaled = scaler_x_train.fit_transform(x_train)

scaler_y_train = StandardScaler()
y_train_scaled = scaler_y_train.fit_transform(y_train)
# scaler_y_train_j2 = StandardScaler()
# y_train_scaled_j2 = scaler_y_train_j2.fit_transform(y_train[:, 1])
# scaler_y_train_j3 = StandardScaler()
# y_train_scaled_j3 = scaler_y_train_j3.fit_transform(y_train[:, 2])

y_train_scaled_j1 = y_train_scaled[:,0]
y_train_scaled_j2 = y_train_scaled[:,1]
y_train_scaled_j3 = y_train_scaled[:,2]

model_j1 = LinearRegression().fit(x_train_scaled, y_train_scaled_j1)
model_j2 = LinearRegression().fit(x_train_scaled, y_train_scaled_j2)
model_j3 = LinearRegression().fit(x_train_scaled, y_train_scaled_j3)
start_time = time.time()

finish_time = time.time() - start_time
print(model_j1.predict(x_train))
print(model_j2.predict(x_train))
print(model_j3.predict(x_train))

r_sq = model_j1.score(x_train_scaled, y_train_scaled_j1)
print('coefficient of determination:', r_sq)
r_sq = model_j2.score(x_train_scaled, y_train_scaled_j2)
print('coefficient of determination:', r_sq)
r_sq = model_j3.score(x_train_scaled, y_train_scaled_j3)
print('coefficient of determination:', r_sq)


In [ ]:
# Evaluate and plot result -----------------------------------------------------
list_finish_time = []
list_infer_time = []
list_rmse_j1_pdt = []
list_rmse_j1_rs_lc = []
list_rmse_j1_rs = []
list_rmse_j2_pdt = []
list_rmse_j2_rs_lc = []
list_rmse_j2_rs = []
list_rmse_j3_pdt = []
list_rmse_j3_rs_lc = []
list_rmse_j3_rs = []

idx = 0
for data_test in data_test_list:
  x_test = data_test[:, feature_select]
  # print(x_test.shape)
  if 12 in feature_select:
    x_test[:,0] = data_test[:, 0] # change the time stamp to relative time (start with 0)
  y_test = data_test[:, 1:4]

  x_test_scaled = scaler_x_train.transform(x_test)

  j1_predict = model_j1.predict(x_test_scaled)
  j2_predict = model_j2.predict(x_test_scaled)
  j3_predict = model_j3.predict(x_test_scaled)

  # print(j1_predict)
  y_predict = np.zeros((x_test.shape[0],3))
  y_predict[:,0] = j1_predict
  y_predict[:,1] = j2_predict
  y_predict[:,2] = j3_predict
  # print(y_predict.shape)
  y_predict = scaler_y_train.inverse_transform(y_predict)

  # load the data for visualization and evaluation only, joint 1 and 2 will be in deg, joint 3 in m
  time_line = data_test[:,0]
  ext_jpos_1 = data_test[:,1] *rad2deg   # joint pose from external joint encoders
  ext_jpos_2 = data_test[:,2] *rad2deg
  ext_jpos_3 = data_test[:,3]

  rs_jpos_1 = data_test[:,13]            # joint pose from ravenstate
  rs_jpos_2 = data_test[:,14]
  rs_jpos_3 = data_test[:,15] *deg2rad

  rs_jpos_1_lc = rs_jpos_1 + bias_jpos_1
  rs_jpos_2_lc = rs_jpos_2 + bias_jpos_2
  rs_jpos_3_lc = rs_jpos_3 + bias_jpos_3

  if train_on_err:
    pdt_jpos_1 = rs_jpos_1 + y_predict[:,0] #*rad2deg
    pdt_jpos_2 = rs_jpos_2 + y_predict[:,1] #*rad2deg
    pdt_jpos_3 = rs_jpos_3 + y_predict[:,2]
  else:
    pdt_jpos_1 = y_predict[:,0] #*rad2deg
    pdt_jpos_2 = y_predict[:,1] #*rad2deg
    pdt_jpos_3 = y_predict[:,2]

  if save_rst:
    pdt_jpos = np.zeros(y_predict.shape)
    pdt_jpos[:,0] = pdt_jpos_1
    pdt_jpos[:,1] = pdt_jpos_2
    pdt_jpos[:,2] = pdt_jpos_3
    np.savetxt('linreg_rst_'+str(idx)+'.csv', pdt_jpos)
  idx += 1

  rmse_j1_pdt = np.sqrt(np.mean(np.square(pdt_jpos_1 - ext_jpos_1)))
  rmse_j2_pdt = np.sqrt(np.mean(np.square(pdt_jpos_2 - ext_jpos_2)))
  rmse_j3_pdt = np.sqrt(np.mean(np.square(pdt_jpos_3 - ext_jpos_3)))
  rmse_j1_rs = np.sqrt(np.mean(np.square(rs_jpos_1 - ext_jpos_1)))
  rmse_j2_rs = np.sqrt(np.mean(np.square(rs_jpos_2 - ext_jpos_2)))
  rmse_j3_rs = np.sqrt(np.mean(np.square(rs_jpos_3 - ext_jpos_3)))
  rmse_j1_rs_lc = np.sqrt(np.mean(np.square(rs_jpos_1_lc - ext_jpos_1)))
  rmse_j2_rs_lc = np.sqrt(np.mean(np.square(rs_jpos_2_lc - ext_jpos_2)))
  rmse_j3_rs_lc = np.sqrt(np.mean(np.square(rs_jpos_3_lc - ext_jpos_3)))

  # testing inference time------------------------------------------------------------------
  # this generator is only for testing the inference time
  infer_start_time = time.time()
  infer_No = 50
  # multi_infer = 25
  # y_predict = np.zeros((x_test.shape[0],3))
  y_predict = [0, 0, 0]
  for i in range(infer_No):
    # _ = model_j1.predict([x_test[i]])
    # _ = model_j2.predict([x_test[i]])
    # _ = model_j3.predict([x_test[i]])
    # print(x_test[i])
    x_test_scaled = scaler_x_train.transform([x_test[i]])
    # print(x_test_scaled)
    y_predict[0] = model_j1.predict(x_test_scaled)
    y_predict[1] = model_j2.predict(x_test_scaled)
    y_predict[2] = model_j3.predict(x_test_scaled)
    # y_predict = scaler_y_train.inverse_transform(y_predict)
  list_infer_time.append((time.time() - infer_start_time)/infer_No)
  #-----------------------------------------------------------------------------------------

  list_finish_time.append(finish_time)
  list_rmse_j1_pdt.append(rmse_j1_pdt)
  list_rmse_j1_rs_lc.append(rmse_j1_rs_lc)
  list_rmse_j1_rs.append(rmse_j1_rs)
  list_rmse_j2_pdt.append(rmse_j2_pdt)
  list_rmse_j2_rs_lc.append(rmse_j2_rs_lc)
  list_rmse_j2_rs.append(rmse_j2_rs)
  list_rmse_j3_pdt.append(rmse_j3_pdt)
  list_rmse_j3_rs_lc.append(rmse_j3_rs_lc)
  list_rmse_j3_rs.append(rmse_j3_rs)


  # fig = plt.figure(figsize=(15, 10))
  # plt.plot(time_line, ext_jpos_1, 'k')
  # plt.plot(time_line, rs_jpos_1, 'b--')
  # plt.plot(time_line, pdt_jpos_1, 'r--')
  # plt.legend(('gt', 'ravenstate', 'learn_cali'))
  # plt.title('Joint 1')
  # plt.xlabel('time(s)')
  # plt.ylabel('Joint Pose (deg)')

  # fig = plt.figure(figsize=(15, 10))
  # plt.plot(time_line, ext_jpos_2, 'k')
  # plt.plot(time_line, rs_jpos_2, 'b--')
  # plt.plot(time_line, pdt_jpos_2, 'r--')
  # plt.legend(('gt', 'ravenstate', 'learn_cali'))
  # plt.title('Joint 2')
  # plt.xlabel('time(s)')
  # plt.ylabel('Joint Pose (deg)')

  # fig = plt.figure(figsize=(15, 10))
  # plt.plot(time_line, ext_jpos_3, 'k')
  # plt.plot(time_line, rs_jpos_3, 'b--')
  # plt.plot(time_line, pdt_jpos_3, 'r--')
  # plt.legend(('gt', 'ravenstate', 'learn_cali'))
  # plt.title('Joint 3')
  # plt.xlabel('time(s)')
  # plt.ylabel('Joint Pose (m)')


  print('-------------------')
  print('Learning Calibration RMSE Joint 1 (deg):')
  print(rmse_j1_pdt)
  print('ravenstate RMSE Joint 1 linear corrected (deg):')
  print(rmse_j1_rs_lc)
  print('ravenstate RMSE Joint 1 (deg):')
  print(rmse_j1_rs)
  print('-------------------')
  print('Learning Calibration RMSE Joint 2 (deg):')
  print(rmse_j2_pdt)
  print('ravenstate RMSE Joint 2 linear corrected (deg):')
  print(rmse_j2_rs_lc)
  print('ravenstate RMSE Joint 2 (deg):')
  print(rmse_j2_rs)
  print('-------------------')
  print('Learning Calibration RMSE Joint 3 (mm):')
  print(rmse_j3_pdt * 1000)
  print('ravenstate RMSE Joint 3 linear corrected (mm):')
  print(rmse_j3_rs_lc * 1000)
  print('ravenstate RMSE Joint 3 (mm):')
  print(rmse_j3_rs * 1000)
  print('-------------------')
  print('|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||')

# print final result ----------------------------------------------------------
print('################################################################################')
print('Final results:')
print('Finish time:')
print(np.mean(list_finish_time))
print(list_finish_time)
print('Inference time:')
print(np.mean(list_infer_time))
print(list_infer_time)
print('-----------------------------------------------')
print('Learning Calibration RMSE Joint 1 (deg):')
for rst in list_rmse_j1_pdt:
  print(rst)
print('ravenstate RMSE Joint 1 linear corrected (deg):')
for rst in list_rmse_j1_rs_lc:
  print(rst)
print('ravenstate RMSE Joint 1 (deg):')
for rst in list_rmse_j1_rs:
  print(rst)

print('-----------------------------------------------')
print('Learning Calibration RMSE Joint 2 (deg):')
for rst in list_rmse_j2_pdt:
  print(rst)
print('ravenstate RMSE Joint 2 linear corrected (deg):')
for rst in list_rmse_j2_rs_lc:
  print(rst)
print('ravenstate RMSE Joint 2 (deg):')
for rst in list_rmse_j2_rs:
  print(rst)

print('-----------------------------------------------')
print('Learning Calibration RMSE Joint 3 (mm):')
for rst in list_rmse_j3_pdt:
  print(rst*1000)
print('ravenstate RMSE Joint 3 linear corrected (mm):')
for rst in list_rmse_j3_rs_lc:
  print(rst*1000)
print('ravenstate RMSE Joint 3 (mm):')
for rst in list_rmse_j3_rs:
  print(rst*1000)
